In [93]:
import pandas as pd
import numpy as np
import urllib.request,sys,time
import requests
from bs4 import BeautifulSoup
import os
import re
from lxml import *
from functools import *

In [94]:
base_url = 'https://web.archive.org/web/20210719095244/https://www.index.hr/'
categories = ['vijesti', 'sport', 'magazin']
categories_url = [base_url + '/' + cat for cat in categories]

In [95]:
articles = np.array([])

for cat in categories:
    url = base_url + '/' + cat
    home_page = requests.get(url).content
    soup = BeautifulSoup(home_page)
    cat_articles = soup.find_all('a', class_= cat + '-text-hover scale-img-hover')
    cat_articles = np.array(['https://web.archive.org' + news['href'] for news in cat_articles])
    articles = np.concatenate([articles, cat_articles], axis = 0)

articles = np.unique(articles)

In [96]:
def read_article(url):
    url_groups = re.search('.*index.hr/(.*)/clanak/(.*)/.*aspx', url)
    title = url_groups.group(2)
    category = url_groups.group(1)
    
    soup = BeautifulSoup(requests.get(url).content)
    content = soup.find_all('div', class_= 'text ' + category + '-link-underline')
    elements = []
    for div in content:
        div_elements = div.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
        elements += div_elements
    elements = [elem.text for elem in elements]
    if len(elements) == 0:
        print(url)
        content = ''
    else:
        content = reduce(lambda s1, s2: s1 + '\n' + s2, elements)
    return [url, title, content, category]

In [97]:
records = list(map(read_article, articles))

In [98]:
dataset = pd.DataFrame.from_records(records, columns = ['url', 'title', 'content', 'category'])
dataset = dataset.sort_values(by=['title'])

In [99]:
dataset.to_csv('index_articles6.csv', index=False)